In [1]:
%matplotlib inline
import warnings
import category_encoders as ce
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.exceptions import DataConversionWarning
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

# Load data
bank = pd.read_csv('bank-additional-full.csv', sep=';')

# Assign to X, y
X = bank.drop(columns='y')
y = bank['y'] == 'yes'

# Drop leaky feature
X = X.drop(columns='duration')

# Split Train, Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Make pipeline
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    StandardScaler(), 
    LogisticRegression(solver='lbfgs', max_iter=1000)
)

In [47]:
from ipywidgets import interact
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, roc_curve

def interact_roc_curve(thresholds=0.5):
    '''
    Relies upon instantiation of X_train, y_train, and 
    pipeline variables prior to execution
    '''
    # Create Predict Proba
    y_pred_proba = cross_val_predict(pipeline, X_train, y_train, cv=3, n_jobs=-1, 
                                 method='predict_proba')[:,1]
    y_pred = y_pred_proba >= thresholds
    
    conf_mat = confusion_matrix(y_train, y_pred)
    true_negative = conf_mat[0][0]
    false_positive = conf_mat[0][1]
    false_negative = conf_mat[1][0]
    true_positive = conf_mat[1][1]
    false_pos_rate = false_positive / (true_negative + false_positive)
    true_pos_rate = true_positive / (true_positive + false_negative)
    
    fpr, tpr, threshold = roc_curve(y_train, y_pred_proba)
    plt.plot(fpr, tpr)
    plt.title('ROC curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    
    plt.scatter(false_pos_rate, true_pos_rate)
    
    plt.show()


interact( interact_roc_curve, 
          thresholds=(0.0,1,0.1));

interactive(children=(FloatSlider(value=0.5, description='thresholds', max=1.0), Output()), _dom_classes=('wid…